In [43]:
import flashalgorithm as fc
import numpy as np
import pickle
import pdb
# water_frac = 0.8
# z_use = np.asarray([
#         [water_frac,(1-water_frac)*0.96,(1-water_frac)*0.04, 0]
#         ])

# z_use = np.asarray([[0.98, 0.02, 0.0, 0.0],
#                     [0.98, 0.0, 0.02, 0.0],
#                     [0.98, 0.019, 0.001, 0],
#                     [0.98, 0.018, 0.002, 0]])
# z_use = np.asarray([[0.98, 0.0199, 0.0001, 0],
#                     [0.98, 0.0198, 0.0002, 0],
#                     [0.98, 0.0004, 0.0196,0],
#                     [0.98, 0.0005, 0.0195,0]])
                    
z_use = np.asarray([[0.98, 0.0001, 0.0199,0],
                    [0.98, 0.0002, 0.0198,0],
                    [0.98, 0.0004, 0.0196,0],
                    [0.98, 0.0005, 0.0195,0]])

# z_use = np.asarray([[0.8, 0.008, 0.192, 0],
#                     [0.8, 0.007, 0.193, 0]])


comp_list = ('water', 'methane', 'ethane', 'propane')
phase_list = ('aqueous', 'vapor', 'lhc', 's1', 's2')
P = 110.0  # bar
T = 273.15 + 18.5 # Kelvin

flash_full = fc.FlashController(components=comp_list,
                                phases=phase_list)
flash_dict = {0: flash_full}
# out_file = str(P) + 'bar' + str(int(T - 273.15)) + 'C_addedcomps_1pt.pkl'
# out_file = str(P) + 'bar' + str(int(T)) + 'T_addedcomps_1pt.pkl'
out_file = 'c1toc3_flashtable_110bar18.5C.pkl'

all_output = []
try:
    all_output = pickle.load(open(out_file, 'rb'))
except:
    all_output =[]
    
def emptycomp_hash(z):
    hashed = sum([2**ii for ii, x in enumerate(z) if x == 0.0])
    return hashed


for ii, z in enumerate(z_use):
    
    comp_hash = emptycomp_hash(z)

    new_comps, new_z = zip(*[
        (comp, z_) for comp, z_ in zip(comp_list, z)
        if z_ != 0.0
    ])
    if comp_hash not in flash_dict.keys():
        flash_dict.update({comp_hash:
                               fc.FlashController(
                                    components=new_comps,
                                    phases=phase_list)})
    flash_use = flash_dict[comp_hash]
    new_z = np.asarray(new_z)
    try:
        initialize_condition = True
        output = flash_use.main_handler(
                    compobjs=flash_use.compobjs,
                    z=new_z,
                    T=T,
                    P=P,
                    initialize=initialize_condition,
                    incipient_calc=False)
        if output[-1] > 1e-6:
            output = flash_use.main_handler(
                compobjs=flash_use.compobjs,
                z=new_z,
                T=T,
                P=P,
                initialize=True,
                incipient_calc=True)
    except:
        try:
            output = flash_use.main_handler(
                        compobjs=flash_use.compobjs,
                        z=new_z,
                        T=T,
                        P=P,
                        initialize=True,
                        incipient_calc=False)
            if output[-1] > 1e-6:
                output = flash_use.main_handler(
                    compobjs=flash_use.compobjs,
                    z=new_z,
                    T=T,
                    P=P,
                    initialize=True,
                    incipient_calc=True)
        except:
            output = []


    all_output.append([ii + len(all_output), z, new_comps, new_z, output])

with open(out_file, 'wb') as f:
     pickle.dump(all_output, f)

In [44]:
all_output[-6:]

[[1168,
  array([ 0.92      ,  0.04421053,  0.03578947,  0.        ]),
  ('water', 'methane', 'ethane'),
  array([ 0.92      ,  0.04421053,  0.03578947]),
  [array([[  9.98049803e-01,   3.09245183e-04,   3.09245183e-04,
             8.57130965e-01,   8.60273843e-01],
          [  1.61841299e-03,   7.50310309e-01,   7.50310308e-01,
             7.85184326e-02,   9.58080737e-02],
          [  3.31783514e-04,   2.49380446e-01,   2.49380447e-01,
             6.43506023e-02,   4.39180835e-02]]),
   array([ 0.44613648,  0.        ,  0.        ,  0.55386352,  0.        ]),
   array([[  1.00000000e+00,   3.05895245e-04,   3.05895245e-04,
             8.58805805e-01,   8.59973742e-01],
          [  1.00000000e+00,   4.57692254e+02,   4.57692254e+02,
             4.85156958e+01,   5.91987793e+01],
          [  1.00000000e+00,   7.42044189e+02,   7.42044190e+02,
             1.93953586e+02,   1.32369698e+02]]),
   12,
   8.0727878789077818e-07]],
 [1169,
  array([ 0.92      ,  0.03529412,  0.0447